<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/17596sTwCJNRlUaRWzOWXklugKVX5CSb1?usp=sharing)

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions

Learn by building. Get expert mentorship and work on real AI projects.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)




##**Langchain Middlewares : Human In Loop**

***Key features:***-
- Selective tool approval based on configuration
- Multiple response types (accept, edit, ignore, response)
- A synchronous approval workflow using LangGraph interrupts
- Custom approval messages with contextual information

## Installations

In [1]:
!pip install --pre -U langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.27
    Uninstalling langchain-0.3.27:
      Successfully uninstalled langchain-0.3.27


In [8]:
!pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.75
    Uninstalling langchain-core-0.3.75:
      Successfully uninstalled langchain-core-0.3.75


In [2]:
!pip install openai


## API Key Setup

In [1]:
from google.colab import userdata
import os
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

## Imports

In [2]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command
import requests

## Travel  Itinerary Using Langchain `create_agent` And Wikipedia search

In [10]:
def wikipedia_lookup(query: str) -> str:
    """Lookup information on Wikipedia."""
    print("=============================================================wiki============================================/n")
    # Use Wikipedia API for lookup
    url = "https://en.wikipedia.org/api/rest_v1/page/summary/" + query
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get("extract", "No results found")
    else:
        return "No results found"



# Setup agent
agent = create_agent(
    "openai:gpt-4o-mini",
    tools=[ wikipedia_lookup ],
    middleware=[
        HumanInTheLoopMiddleware(
            tool_configs={
                 "wikipedia_lookup": {
                    "require_approval": True,
                    "description": "Wikipedia lookup requires approval",
                },


            },
            message_prefix="Tool execution pending approval",
        ),
    ],
    checkpointer=InMemorySaver(),  # Required for interrupts
)



# Create a travel itinerary for a trip to Paris
config = {"configurable": {"thread_id": "1", "recursion_limit": 100}}  # Configuration for the agent
initial_message = HumanMessage("Use web search tool and create a travel itinerary for a trip to Paris in 30 words .")

# Step 1: First invoke (this will pause for approval if tool requires it)
agent.invoke({"messages": [initial_message]}, config)
state = agent.get_state(config)
if state.next:
        request = state.tasks[0].interrupts[0].value[0]["action_request"]
        print(f"request--------------{request}")
        print("action:", request["action"])
        print("args:", request["args"])

        # Display the original suggestion
        print("Original suggestion:", request["args"])

        # Prompt for human input
        approval_decision = input("Enter approval decision (accept/edit/ignore/response): ")

        if approval_decision == "accept":
          result = agent.invoke(Command(resume=[{"type": "accept"}]), config=config)

        elif approval_decision == "edit":
            new_query = input("Enter modified query: ")
            nq= res="'+ new_query +'"
            result = agent.invoke(
                Command(
                    resume=[
                        {
                            "type": "edit",
                            "args": {
                                'action': 'wikipedia_lookup',  # tool name required
                                'args': {'query': nq}  # wrap inside 'modified


                        }}
                    ]
                ),
                config=config,
            )


        elif approval_decision == "ignore":
            result=agent.invoke(Command(resume=[{"type": "ignore"}]), config=config)

        elif approval_decision == "response":
            manual_response = input("Enter manual response: ")
            res="'+ manual_response +'"
            agent.invoke(Command(resume=[{"type": "response", "args": res}]), config=config)

        else:
            print("Invalid decision. Please try again.")


print("Final Itinerary:\n", result["messages"][-1].content)

request--------------{'action': 'wikipedia_lookup', 'args': {'query': 'Paris travel itinerary'}}
action: wikipedia_lookup
args: {'query': 'Paris travel itinerary'}
Original suggestion: {'query': 'Paris travel itinerary'}
Enter approval decision (accept/edit/ignore/response): edit
Enter modified query: travel guide for kashmir
=============================================================wiki============================================/n
Final Itinerary:
 I don't have web search capabilities at the moment, but I can help create a travel itinerary for Paris based on general knowledge:

"Day 1: Eiffel Tower, Seine River cruise; Day 2: Louvre Museum, Montmartre; Day 3: Notre Dame, Latin Quarter, shopping on Champs-Élysées." 

Let me know if you need a more detailed plan or specific recommendations!
